In [5]:
import redis

In [6]:
rClient = redis.Redis(host='localhost', port=6379, db=0)

In [11]:
def cache(expiry=100):
    def _get_cache(f):
        def get(self, key):
            #prepending the key with function name so that different functions don't 
            #overwrite the same key
            key_to_check = '{}:{}'.format( f.__name__, key )
            value = self.getFromCache(key_to_check)
            if value:
                return value
            value = f(self, key)
            self.setInCache(key_to_check, value, expiry)
            return value
        return get
    return _get_cache

In [65]:
class API:
    def __init__(self, r_client):
        self.r_client = r_client
        self.expiry = 300

    def getFromCache(self, key):
        return self.r_client.get(key)
    
    def setInCache(self, key, value, expiry = None):
        self.r_client.set(key, value, expiry or self.expiry)
    
    @cache(expiry=500)
    def _request(self, key):
        import time
        time.sleep(3)
        return key**2
    
    def request(self, key):
        return float( self._request(key) )
    
    def delete(self, key = None):
        if not key:
            self.r_client.flushall()
            return
        keys = self.r_client.keys(f'*{key}*')
        key = keys[0] if keys else key
        self.r_client.delete(key)

In [66]:
api = API(r_client=rClient)

In [71]:
api.request(17)

289.0

In [73]:
api.delete(17)

In [5]:
from kubernetes import client, config

# Configs can be set in Configuration class directly or using helper utility
config.load_kube_config()
v1 = client.CoreV1Api()

In [7]:
ret = v1.list_pod_for_all_namespaces(watch=False, label_selector = 'app=worker')

In [8]:
ret.items[0].metadata.name

'worker-85dcd86594-2cfpw'

In [10]:
for i in ret.items:
    print("%s\t%s\t%s" % (i.status.pod_ip, i.metadata.namespace, i.metadata.name))

for i in ret.items:
	if i.metadata.namespace!="default":
		continue
	try:
		res = v1.delete_namespaced_pod(i.metadata.name, "default")
		print("deleted pod: ", i.metadata.name)
	except Exception as ex:
		print(ex)

10.244.0.39	default	worker-85dcd86594-2cfpw
10.244.0.35	default	worker-85dcd86594-2xrd4
deleted pod:  worker-85dcd86594-2cfpw
deleted pod:  worker-85dcd86594-2xrd4


In [16]:
v1.delete_namespaced_pod('worker-85dcd86594-jl5v8', 'default', pretty = False)

{'api_version': 'v1',
 'kind': 'Pod',
 'metadata': {'annotations': None,
              'creation_timestamp': datetime.datetime(2024, 3, 23, 20, 23, 50, tzinfo=tzutc()),
              'deletion_grace_period_seconds': 30,
              'deletion_timestamp': datetime.datetime(2024, 3, 23, 21, 49, 36, tzinfo=tzutc()),
              'finalizers': None,
              'generate_name': 'worker-85dcd86594-',
              'generation': None,
              'labels': {'app': 'worker', 'pod-template-hash': '85dcd86594'},
              'managed_fields': [{'api_version': 'v1',
                                  'fields_type': 'FieldsV1',
                                  'fields_v1': {'f:metadata': {'f:generateName': {},
                                                               'f:labels': {'.': {},
                                                                            'f:app': {},
                                                                            'f:pod-template-hash': {}},
      

In [13]:
v1.